# Capstone: Data Collection with Webscraping

In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np

### Helper Functions

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

## TASK 1: 
* Request the Falcon9 Launch Wiki page from its URL

In [4]:
# use requests.get() method with the provided static_url
# assign the response to a object
response = requests.get(static_url)
response.status_code

200

In [5]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html5lib')

In [6]:
title = str(soup.title)
title = title.replace('<title>', '')
title = title.replace('</title>', '')
print(title)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


## TASK 2: 
* Extract all column/variable names from the HTML table header

In [7]:
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

In [8]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)

In [9]:
table_headers = first_launch_table.find_all('th')
table_headers

[<th scope="col">Flight No.
 </th>,
 <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
 </th>,
 <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
 </th>,
 <th scope="col">Launch site
 </th>,
 <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
 </th>,
 <th scope="col">Payload mass
 </th>,
 <th scope="col">Orbit
 </th>,
 <th scope="col">Customer
 </th>,
 <th scope="col">Launch<br/>outcome
 </th>,
 <th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">1
 </th>,
 <th rowspan="2" scope="row" style="text-align:center;">2
 </th>,
 <

In [10]:
column_names = []

#Get all headers from the table
for header in first_launch_table.find_all('th'): 
    column_name = extract_column_from_header(header)
    if (column_name is not None) and (len(column_name) > 0):
        column_names = np.append(column_names, column_name)    
    
print(column_names)

['Flight No.' 'Date and time ( )' 'Launch site' 'Payload' 'Payload mass'
 'Orbit' 'Customer' 'Launch outcome']


## TASK 3: 
* Create a data frame by parsing the launch HTML tables

In [42]:
launch_dict = dict.fromkeys(column_names, np.array([]))
launch_dict

{'Flight No.': array([], dtype=float64),
 'Date and time ( )': array([], dtype=float64),
 'Launch site': array([], dtype=float64),
 'Payload': array([], dtype=float64),
 'Payload mass': array([], dtype=float64),
 'Orbit': array([], dtype=float64),
 'Customer': array([], dtype=float64),
 'Launch outcome': array([], dtype=float64)}

In [43]:
# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Added some new columns
launch_dict['Version Booster'] = np.array([])
launch_dict['Booster landing'] = np.array([])
launch_dict['Date'] = np.array([])
launch_dict['Time'] = np.array([])

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.

In [44]:
extracted_row = 0

#Extract each table 
for table_number, table in enumerate(soup.find_all('table',
                                                   "wikitable plainrowheaders collapsible")):
    
   # get table row 
    for rows in table.find_all("tr"):
        
        #check to see if first table heading is as 
        # number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
            
        #get table element 
        row = rows.find_all('td')
        
        #if it is number save cells in a dictonary 
        if flag:
            
            extracted_row += 1
            
            # Flight Number value
            # TODO: Append the flight_number into launch_dict 
            # with key `Flight No.`
            launch_dict['Flight No.'] = np.append(launch_dict['Flight No.'],
                                                  flight_number) 
            #print('A. Flight No:', flight_number)
            
            datatimelist = date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            launch_dict['Date'] = np.append(launch_dict['Date'], date) 
            #print('B. Date:', date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            launch_dict['Time']= np.append(launch_dict['Time'], time)
            #print('C. Time:', time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string
            launch_dict['Version Booster'] = np.append(launch_dict['Version Booster'],
                                                       bv)
            #print('D. Booster Version:', bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch site`
            launch_site = row[2].a.string
            launch_dict['Launch site'] = np.append(launch_dict['Launch site'], 
                                                   launch_site)
            #print('E. Launch Site:', launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'] = np.append(launch_dict['Payload'], payload)
            #print('F. Payload:', payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'] = np.append(launch_dict['Payload mass'],
                                                    payload_mass)
            #print('G. Payload Mass:', payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'] = np.append(launch_dict['Orbit'], orbit)
            #print('H. Orbit:', orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6]
            try:
                customer = customer.a.string
            except AttributeError:
                print('')
            launch_dict['Customer'] = np.append(launch_dict['Customer'], customer)
            #print('I. Customer:', customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'] = np.append(launch_dict['Launch outcome'],
                                                      launch_outcome)
            #print('J. Launch outcome:', launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'] = np.append(launch_dict['Booster landing'],
                                                       booster_landing)
            #print('K. Booster Landing:', booster_landing)
            
            #print()

In [23]:
def printColumns():
        print('Flight No.', len(launch_dict['Flight No.']))
        print('Launch site', len(launch_dict['Launch site']))
        print('Payload', len(launch_dict['Payload']))
        print('Payload mass', len(launch_dict['Payload mass']))
        print('Orbit', len(launch_dict['Orbit']))
        print('Customer', len(launch_dict['Customer']))
        print('Launch outcome', len(launch_dict['Launch outcome']))
        print('Version Booster', len(launch_dict['Version Booster']))
        print('Booster landing', len(launch_dict['Booster landing']))
        print('Date', len(launch_dict['Date']))
        print('Time', len(launch_dict['Time']))

In [45]:
printColumns()

Flight No. 121
Launch site 121
Payload 121
Payload mass 121
Orbit 121
Customer 121
Launch outcome 121
Version Booster 121
Booster landing 121
Date 121
Time 121


In [46]:
# Save to a DataFrame
df = pd.DataFrame(launch_dict)
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0.0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
116,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1051.10,Success,9 May 2021,06:42
117,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,F9 B5B1058.8,Success,15 May 2021,22:56
118,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5B1063.2,Success,26 May 2021,18:59
119,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,F9 B5B1067.1,Success,3 June 2021,17:29


In [47]:
# Save DataFrame to CSV file
df.to_csv('spacex_web_scraped.csv', index=False)